In [2]:
# import statements
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from lxml import html
from bs4 import BeautifulSoup
import requests
import time
import re

In [3]:
df = pd.read_csv("../../data/youtube_data/prin_polly_youtube_data.csv")
df.head(2)

,Title,Links
0,updating my WHOLE WARDROBE for summer *princes...,http://www.princesspolly.com.au\nhttps://www.p...
1,ruthlessly ranking IT GIRL brands (Réalisation...,https://poshmark.com/closet/alexasunshine83\nh...


In [98]:
def scrape_princess_polly(url):
    try:
        # Set Chrome options for headless mode
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        # Initialize the WebDriver
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(url)
        # Get the page source after interactions
        page_source = driver.page_source
        # Close the WebDriver once done to avoid multiple instances
        driver.quit()
        # Parse the page source with Beautiful Soup
        soup = BeautifulSoup(page_source, "html.parser")

        # get name of product
        title_element = soup.find("h1", class_="product__title")
        title = title_element.text.strip()
 
        description_list = soup.find("div", class_="product-details__content-inner")
        description_items = description_list.text.split('\n')
        description_items = [item for item in description_items if item.find('%') != -1]
        
        composition = (description_items[0]).split(',')[0]
        if composition.find(': ') != -1:
           composition = composition.split(': ')[1]

        pattern = r'(\d+)%\s*([\w\s-]+?(?=\s*\d+%|$))'
        fabric_percentages = re.findall(pattern, composition)

        materials = {}

        for percent, fabric in fabric_percentages:
            fabric = fabric.lower()
            fabric = fabric.replace('\xa0', '')
            materials[fabric] = percent

        final_data = {'item': title}
        final_data.update(materials)

        return final_data
        # else:
        #     return "Data element not found on the page."
    except Exception as e:
        None

In [93]:
# URL of the webpage you want to scrape
url = 'https://us.princesspolly.com/products/recycled-nylon-layla-bikini-bottoms-brown?variant=39501925318740' # 80% reclaimed nylon 20% spandex

# Call the function to scrape the website
data = scrape_princess_polly(url)
print(data)

{'item': 'Layla high cut bikini bottoms brown', 'reclaimed nylon': '80', 'spandex': '20'}


In [99]:
def scrape_and_update(row):
    links_list = row["Links"].split('\n')
    scraped_data_combined = ""

    for link in links_list:
        print(f"Scraping data from link: {link}")
        scraped_data = scrape_princess_polly(link)
        if scraped_data:
            scraped_data_combined += str(scraped_data) + "\n"
            print(scraped_data)
        time.sleep(3)
        print()

    return scraped_data_combined

In [100]:
df['ScrapedData'] = df.apply(scrape_and_update, axis=1)

Scraping data from link: http://www.princesspolly.com.au



Scraping data from link: https://www.princesspolly.com.au/products/kayte-scrunchie-white](https://www.princesspolly.com.au/products/kayte-scrunchie-white

Scraping data from link: https://www.princesspolly.com.au/products/kendelton-hair-clip-pack-white](https://www.princesspolly.com.au/products/kendelton-hair-clip-pack-white

Scraping data from link: https://www.princesspolly.com.au/products/katrine-flower-maxi-dress-black?variant=39903861407816](https://www.princesspolly.com.au/products/katrine-flower-maxi-dress-black?variant=39903861407816

Scraping data from link: https://www.princesspolly.com.au/products/barely-there-tie-top-onyx?variant=39956051329096](https://www.princesspolly.com.au/products/barely-there-tie-top-onyx?variant=39956051329096
{'item': 'Barely there tie top onyx', 'polyester': '100'}

Scraping data from link: https://www.princesspolly.com.au/products/belle-knit-maxi-skirt-white-petite?variant=39878123323464](https://www.princesspolly.com.au/products/belle-knit-maxi

KeyboardInterrupt: 